In [2]:
import pandas as pd
import numpy as np

import imblearn #deal w/ imbalance dataset mainly

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

In [3]:
churn = pd.read_csv("/Users/giacomorossini/Desktop/IRONHACK/Laboratory/LAB10/lab-imbalanced-data/files_for_lab/customer_churn.csv")
cols=[]
for i in range(len(churn.columns)):
    cols.append(churn.columns[i].lower().replace(' ','_'))
    
churn.columns=cols

In [4]:
churn.groupby('churn').count() #no missing data, imbalanced confirmed more 0 than 1

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
churn,,,,,,,,,,,,,,,,,,,,
No,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174,5174
Yes,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869,1869


In [8]:
#2)&3) extract my target and filter my DB
churn_analysis = churn[['churn', 'tenure', 'seniorcitizen','monthlycharges']]
churn_analysis['churn']=churn_analysis['churn'].replace({'Yes': 1, 'No': 0}) #replacing my Y to get 1 for 'YES' & 0 for 'No'

#4)normalizing my data

from sklearn.preprocessing import MinMaxScaler

num_data = churn_analysis.drop(['churn'], axis=1)
scaler = MinMaxScaler()
normalized_churn = scaler.fit_transform(num_data)
normalized_churn = pd.DataFrame(normalized_churn, columns=num_data.columns)
normalized_churn['churn']=churn_analysis['churn']
normalized_churn

,tenure,seniorcitizen,monthlycharges,churn
0,0.013889,0.0,0.115423,0
1,0.472222,0.0,0.385075,0
2,0.027778,0.0,0.354229,1
3,0.625000,0.0,0.239303,0
4,0.027778,0.0,0.521891,1
...,...,...,...,...
7038,0.333333,0.0,0.662189,0
7039,1.000000,0.0,0.845274,0
7040,0.152778,0.0,0.112935,0
7041,0.055556,1.0,0.558706,1


In [9]:
#5)building logistic regression

X = normalized_churn.drop('churn', axis=1)
Y = normalized_churn['churn']

X_train , X_test , Y_train , Y_test = train_test_split (X , Y , random_state=42, test_size = 0.2)

LR = LogisticRegression()
LR.fit(X_train,Y_train)
LR.score(X_test,Y_test)

0.8041163946061036

In [10]:
#6)Evaluating the model using 6 metrics:

pred = LR.predict(X_test)
print("1)Precision is: ")
print(precision_score(Y_test, pred))
print("2)Recall is:")
print(recall_score(Y_test, pred))
print("3)F1 is:")
print(f1_score(Y_test, pred))
print("4)Classification_report:")
print(classification_report(Y_test, pred))
print("5)Confusion matrix:")
print(confusion_matrix(Y_test, pred))

1)Precision is: 
0.6932270916334662
2)Recall is:
0.46648793565683644
3)F1 is:
0.5576923076923076
4)Classification_report:
              precision    recall  f1-score   support

           0       0.83      0.93      0.87      1036
           1       0.69      0.47      0.56       373

    accuracy                           0.80      1409
   macro avg       0.76      0.70      0.72      1409
weighted avg       0.79      0.80      0.79      1409

5)Confusion matrix:
[[959  77]
 [199 174]]


In [14]:
#7) My accuracy is high because i have inbalance data, so my 'No' brought this value on highest ('No'>>>'Yes')

In [17]:
#8) Applying SMOTE

sm = SMOTE(random_state = 0, sampling_strategy =1.0)

X_train_SMOTE, Y_train_SMOTE = sm.fit_resample(X_train, Y_train)

In [21]:
train_smote = pd.concat ([X_train_SMOTE,Y_train_SMOTE], axis = 0)

LR = LogisticRegression (max_iter = 30)

LR.fit(X_train_SMOTE,Y_train_SMOTE)
pred = LR.predict(X_test)
print("1)Precision is: ")
print(precision_score(Y_test, pred))
print("2)Recall is:")
print(recall_score(Y_test, pred))
print("3)F1 is:")
print(f1_score(Y_test, pred))
print("4)Classification_report:")
print(classification_report(Y_test, pred))
print("5)Confusion matrix:")
print(confusion_matrix(Y_test, pred))

1)Precision is: 
0.5107142857142857
2)Recall is:
0.7667560321715817
3)F1 is:
0.6130760986066451
4)Classification_report:
              precision    recall  f1-score   support

           0       0.90      0.74      0.81      1036
           1       0.51      0.77      0.61       373

    accuracy                           0.74      1409
   macro avg       0.70      0.75      0.71      1409
weighted avg       0.80      0.74      0.76      1409

5)Confusion matrix:
[[762 274]
 [ 87 286]]


In [ ]:
'''
8) Applying SMOTE i had a drop in acc & precision but a significant improve in recall - Evaluating these metrics
in this specific topic, my interest is to catch all the cases of 'churn' happened so i have improved cases where 
my prediction where 0 but the client was 1

'''